This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Beyond text classification: Sequence-to-sequence learning

### A machine translation example

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-11-09 20:24:08--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
storage.googleapis.com (storage.googleapis.com) 해석 중... 142.251.130.27, 142.250.66.59, 172.217.24.91, ...
다음으로 연결 중: storage.googleapis.com (storage.googleapis.com)|142.251.130.27|:80... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 2638744 (2.5M) [application/zip]
저장 위치: `spa-eng.zip'

spa-eng.zip         100%[===================>]   2.52M  4.98MB/s    /  0.5s    

2023-11-09 20:24:09 (4.98 MB/s) - `spa-eng.zip' 저장함 [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

('She made friends with him in Boston.', '[start] Ella se hizo amiga suya en Boston. [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [5]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

2023-11-09 20:24:43.757410: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-09 20:24:43.757724: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-09 20:24:43.974623: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-09 20:24:44.034478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:24:46.791237: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


**Preparing datasets for the translation task**

In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2023-11-09 20:24:49.613519: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Sequence-to-sequence learning with RNNs

**GRU-based encoder**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

**GRU-based decoder and the end-to-end model**

In [9]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [10]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2023-11-09 20:24:55.577244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:24:56.834638: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}

	while inferring type of node 'cond_41/output/_23'
2023-11-09 20:24:56.838685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:24:56.838756: I tens

1302/1302 [==============================] - ETA: 0s - loss: 1.5524 - accuracy: 0.4622

2023-11-09 20:29:13.966047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:29:14.580256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:29:14.648075: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 20:29:14.776832: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1302/1302 [==============================] - 281s 211ms/step - loss: 1.5524 - accuracy: 0.4622 - val_loss: 1.1327 - val_accuracy: 0.6137
Epoch 2/15
1302/1302 [==============================] - 272s 209ms/step - loss: 1.0841 - accuracy: 0.6381 - val_loss: 0.9595 - val_accuracy: 0.6884
Epoch 3/15
1302/1302 [==============================] - 273s 210ms/step - loss: 0.9355 - accuracy: 0.6778 - val_loss: 0.9132 - val_accuracy: 0.7100
Epoch 4/15
1302/1302 [==============================] - 272s 209ms/step - loss: 0.8882 - accuracy: 0.6897 - val_loss: 0.9424 - val_accuracy: 0.7031
Epoch 5/15
1302/1302 [==============================] - 272s 209ms/step - loss: 0.8973 - accuracy: 0.6921 - val_loss: 0.9301 - val_accuracy: 0.7114
Epoch 6/15
1302/1302 [==============================] - 275s 211ms/step - loss: 0.9465 - accuracy: 0.6880 - val_loss: 0.9275 - val_accuracy: 0.7132
Epoch 7/15
1302/1302 [==============================] - 274s 210ms/step - loss: 0.9908 - accuracy: 0.6825 - val_loss: 0.933

**Translating new sentences with our RNN encoder and decoder**

In [11]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I've only been online for ten minutes.


2023-11-09 21:33:47.836357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 21:33:48.425473: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 21:33:48.498932: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-11-09 21:33:49.077328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 33ms/step
[start] no cómo no en no en no en no en no en no en no en no en no en
-
It's not easy to speak a foreign language.
1/1 [==============================] - 0s 33ms/step
[start] no cómo no cómo no cómo no cómo no cómo no cómo no cómo no les no les no les
-
He decided on the red car.
1/1 [==============================] - 0s 39ms/step
[start] la cómo la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela
-
Excuse my clumsiness.
1/1 [==============================] - 0s 37ms/step
[start] la cómo la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela
-
Tom realized that he had lost his wallet.
1/1 [==============================] - 0s 32ms/step
[start] la cómo la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela la escuela
-
Tom says he never lies to Mary, but he often does.
1/1 [==============================] - 0s 37m

### Sequence-to-sequence learning with Transformer

#### The Transformer encoder

In [17]:
class TransformerEncoder(layers.Layer):  
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention( 
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),  
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()  
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

#### The Transformer decoder

In [18]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [19]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [20]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

In [21]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


2023-11-12 14:53:25.628344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1302/1302 [==============================] - ETA: 0s - loss: 1.6537 - accuracy: 0.4250

2023-11-12 14:59:37.525345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1302/1302 [==============================] - 409s 311ms/step - loss: 1.6537 - accuracy: 0.4250 - val_loss: 1.3183 - val_accuracy: 0.5182
Epoch 2/30
1302/1302 [==============================] - 403s 309ms/step - loss: 1.2950 - accuracy: 0.5443 - val_loss: 1.1616 - val_accuracy: 0.5789
Epoch 3/30
1302/1302 [==============================] - 406s 312ms/step - loss: 1.1347 - accuracy: 0.5957 - val_loss: 1.0627 - val_accuracy: 0.6121
Epoch 4/30
1302/1302 [==============================] - 392s 301ms/step - loss: 1.0447 - accuracy: 0.6285 - val_loss: 1.0298 - val_accuracy: 0.6267
Epoch 5/30
1302/1302 [==============================] - 393s 302ms/step - loss: 0.9956 - accuracy: 0.6517 - val_loss: 1.0104 - val_accuracy: 0.6358
Epoch 6/30
1302/1302 [==============================] - 400s 307ms/step - loss: 0.9601 - accuracy: 0.6697 - val_loss: 0.9937 - val_accuracy: 0.6456
Epoch 7/30
1302/1302 [==============================] - 399s 307ms/step - loss: 0.9330 - accuracy: 0.6838 - val_loss: 0.994

**Translating new sentences with our Transformer model**

In [22]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Why is Tom there?
[start] por qué está allí tom [end]
-
I ran around the field.
[start] yo me he dejado de por el campo [end]
-
He took my umbrella without bothering to ask.
[start] se sacó el paraguas sin que sin pedirle agua [end]
-
The house is on the most level part of the ground.
[start] la casa está al hombre más parte del ahora del piso [end]
-
She watched him and the other boys playing baseball.
[start] ella le vio y el otro béisbol es juego de jugar al béisbol [end]
-
From there, one could see perfectly.
[start] desde un día podía ver a la discusión [end]
-
The boy said that the taxi vanished into the fog.
[start] el niño dijo que el parque [UNK] que [UNK] en la cena [end]
-
I am good.
[start] estoy bueno [end]
-
Open the box.
[start] cierra la caja [end]
-
Tom is behind you.
[start] tom está detrás de ti [end]
-
Don't make any loud noises.
[start] no hagas daño el [UNK] [end]
-
Hebrew is my mother tongue.
[start] la [UNK] es mi lengua la lengua de edad [end]
-
The hard work

### Additional Experiment

In [24]:
from tensorflow.keras.optimizers import Adam

### change hyperparameter values
embed_dim = 256  # (25 -> 256)
dense_dim = 2048  # (1024 -> 2048)
num_heads = 6  # (8 -> 6)

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.4)(x)  # Dropout (0.3 -> 0.4)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
transformer.compile(
    optimizer=Adam(learning_rate=1e-3),  # learning rate (R -> 1e-3)
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30


2023-11-12 21:09:58.690459: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1302/1302 [==============================] - ETA: 0s - loss: 1.3238 - accuracy: 0.4939

2023-11-12 21:15:11.413929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1302/1302 [==============================] - 346s 264ms/step - loss: 1.3238 - accuracy: 0.4939 - val_loss: 0.8823 - val_accuracy: 0.6098
Epoch 2/30
1302/1302 [==============================] - 343s 263ms/step - loss: 0.6980 - accuracy: 0.6534 - val_loss: 0.7362 - val_accuracy: 0.6533
Epoch 3/30
1302/1302 [==============================] - 340s 261ms/step - loss: 0.4463 - accuracy: 0.7454 - val_loss: 0.7154 - val_accuracy: 0.6632
Epoch 4/30
1302/1302 [==============================] - 341s 262ms/step - loss: 0.3264 - accuracy: 0.8023 - val_loss: 0.7345 - val_accuracy: 0.6708
Epoch 5/30
1302/1302 [==============================] - 340s 261ms/step - loss: 0.2611 - accuracy: 0.8349 - val_loss: 0.7723 - val_accuracy: 0.6723
Epoch 6/30
1302/1302 [==============================] - 340s 262ms/step - loss: 0.2190 - accuracy: 0.8561 - val_loss: 0.8228 - val_accuracy: 0.6694
Epoch 7/30
1302/1302 [==============================] - 340s 261ms/step - loss: 0.1898 - accuracy: 0.8718 - val_loss: 0.871

In [25]:
# Test Translating
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence]) 
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts) 
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
He paid $20 for the lipstick.
[start] Él pagó se levantó por la [UNK] [end]
-
She advised him to stay in bed for two more days.
[start] le aconsejó que se [UNK] en la cama a dos días [end]
-
A fire broke out in the supermarket last night.
[start] un incendio en el vecindario la noche [end]
-
She is a very intelligent young lady.
[start] ella es una persona muy joven [end]
-
World War I broke out in 1914.
[start] la guerra se me da pasando bien [end]
-
I've kept my weight down even though many of my friends have gained weight as they've grown older.
[start] he tomado el peso de pesar de tener muchos amigos que mi propio peso [end]
-
That's very dangerous.
[start] eso es muy peligroso [end]
-
I've never been spoken to like that.
[start] nunca he hablado como eso [end]
-
I shouldn't have to do all this work by myself.
[start] no debí tener que hacer todo el trabajo yo solo [end]
-
Don't judge people by their appearance.
[start] no [UNK] a la gente de su apariencia [end]
-
Let's wait her